In [ ]:
from pyecospold import parse_file_v2
import pandas as pd
import os
from datetime import datetime
import logging
import csv

# Global mapping dictionary from activity_id to (shortname, activityName)
global_activity_mapping = {}

# Initialize statistical variables
total_files = 0
converted_files = 0
failed_files = 0
failed_file_names = []

# Used to track the number of deleted rows in each file
deleted_rows_per_file = {}

# Used to track the replaced process_name in each file
replaced_process_name = {}

# Used to count the number of rows where activityLinkId is missing and amount is not 1
non1_amount_per_file = {}

# Used to count the number of rows where activityLinkId is missing and amount is -1
neg1_amount_per_file = {}

# Used to record files that still have [Unknown Location]Unknown Activity Name
unknown_activity_files = []

# Define input and output folder paths
input_folder = "C:\\Users\\WasteWang\\LCA\\DATA\\3.11_APOS\\datasets"
output_folder_base = "C:\\Users\\WasteWang\\LCA\\OUTPUT"
lookup_file_path = "C:\\Users\\WasteWang\\LCA\\DATA\\3.11_APOS\\FilenameToActivityLookup.csv"
batch_file_path = "C:\\Users\\WasteWang\\LCA\\batch_number.txt"  # File path for storing the batch number

# If the batch number file does not exist, initialize it to 1
if not os.path.exists(batch_file_path):
    try:
        with open(batch_file_path, "w") as f:
            f.write("1")
        print("Batch number file created with initial value 1.")
    except Exception as e:
        print(f"Error creating batch number file: {e}")

# Read the batch number
try:
    with open(batch_file_path, "r") as f:
        batch_number = int(f.read().strip())
    print(f"Current batch number: {batch_number}")
except Exception as e:
    print(f"Error reading batch number file: {e}")
    batch_number = 1  # Default value

# Get the current date
current_date = datetime.now().strftime("%m%d")
print(f"Current date: {current_date}")

# Create the output folder for the current batch
batch_folder = os.path.join(output_folder_base, f"{current_date}_{batch_number}")
os.makedirs(batch_folder, exist_ok=True)
print(f"Output will be saved to: {batch_folder}")

# Configure logging, output to file only, set to INFO level
logger = logging.getLogger('spold_processor')
logger.setLevel(logging.INFO)

# Create a file handler to save to processing_debug.txt in the output batch folder
file_handler = logging.FileHandler(os.path.join(batch_folder, 'processing_debug.txt'), encoding='utf-8')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s %(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Load the mapping table from filename to activity name, specify semicolon delimiter
try:
    lookup_df = pd.read_csv(lookup_file_path, sep=';')
    lookup_dict = {row['Filename'].split('_')[0]: (row['ActivityName'], row['Location']) for _, row in lookup_df.iterrows()}
    logger.info(f"Loaded lookup dictionary with {len(lookup_dict)} entries.")
except Exception as e:
    logger.error(f"Error loading lookup file: {e}")
    lookup_dict = {}

# Verify the integrity of lookup_dict
def verify_lookup_dict():
    missing_prefixes = set()
    for filename in os.listdir(input_folder):
        if filename.endswith(".spold"):
            prefix = filename.split('_')[0] if '_' in filename else os.path.splitext(filename)[0]
            if prefix not in lookup_dict:
                missing_prefixes.add(prefix)
    if missing_prefixes:
        logger.warning(f"The following prefixes are missing in lookup_dict: {', '.join(missing_prefixes)}")
    else:
        logger.info("All prefixes are present in lookup_dict.")

verify_lookup_dict()

def extract_activity_mapping(file_path):
    """
    Extracts all activityDescription elements from a single .spold file and returns a mapping dictionary.
    """
    mapping = {}
    try:
        logger.info(f"Parsing file: {file_path}")
        ecoSpold = parse_file_v2(file_path)
        namespaces = {'eco': 'http://www.EcoInvent.org/EcoSpold02'}
        
        # Get all activityDescription elements, including sub-activities
        activity_descriptions = ecoSpold.findall('.//eco:activityDescription', namespaces)
        for activity_description in activity_descriptions:
            activity = activity_description.find('eco:activity', namespaces)
            if activity is not None:
                activity_id = activity.attrib.get('id')
                activity_name_elem = activity.find('eco:activityName', namespaces)
                activity_name_text = activity_name_elem.text.strip() if activity_name_elem is not None and activity_name_elem.text else "Unknown Activity Name"

                geography = activity_description.find('eco:geography', namespaces)
                if geography is not None:
                    shortname_elem = geography.find('eco:shortname', namespaces)
                    shortname = shortname_elem.text.strip() if shortname_elem is not None and shortname_elem.text else "Unknown Location"
                else:
                    shortname = "Unknown Location"

                if activity_id:
                    mapping[activity_id] = (shortname, activity_name_text)
                    logger.info(f"Mapped activity_id {activity_id} to ({shortname}, {activity_name_text})")
    except Exception as e:
        logger.error(f"Error extracting activity mapping from file {file_path}: {e}")
    return mapping

def process_activity_description(activityDescription, current_activity_name, namespaces):
    """
    Processes the information in the activityDescription section and returns a list of dictionary records.
    """
    records = []
    try:
        if activityDescription is not None:
            for field, tag in [('includedActivitiesStart', 'includedActivitiesStart'),
                               ('includedActivitiesEnd', 'includedActivitiesEnd'),
                               ('generalComment', 'generalComment')]:
                value = activityDescription.find(f'eco:{tag}', namespaces)
                value_text = value.text.strip() if value is not None and value.text else 'N/A'
                category = 'Included Activities' if field != 'generalComment' else 'General Comment'
                record = {
                    'process_name': current_activity_name,
                    'flow': '',  # This field has no data in this section
                    'unit': '',
                    'amount': '',
                    'category': category,
                    'field': field,
                    'value': value_text,
                    'compartment': '',
                    'subcompartment': '',
                    'comment': '',
                    'outputGroup': '',
                    'section': 'activityDescription',
                    'activityLinkId': '',
                    'intermediateExchangeId': ''
                }
                records.append(record)
    except Exception as e:
        logger.error(f"Error processing activityDescription: {e}")
    return records

def process_intermediate_exchange(exchange, current_activity_name, current_location, filename, file_activity_mapping, namespaces):
    """
    Processes an intermediateExchange element and returns a dictionary record.
    """
    try:
        intermediateExchangeId = exchange.attrib.get('intermediateExchangeId', '').strip()
        amount_str = exchange.attrib.get('amount', 'N/A').strip()
        try:
            amount = float(amount_str)
        except ValueError:
            amount = None
            logger.warning(f"{filename}: Invalid amount value '{amount_str}' in intermediateExchangeId '{intermediateExchangeId}'.")
        
        flow_name_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}name')
        flow_name = flow_name_elem.text.strip() if flow_name_elem is not None and flow_name_elem.text else "Unknown Flow"
        unit_name_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}unitName')
        unit_name = unit_name_elem.text.strip() if unit_name_elem is not None and unit_name_elem.text else "Unknown Unit"
        comment_field = exchange.find('{http://www.EcoInvent.org/EcoSpold02}comment')
        comment = comment_field.text.strip() if comment_field is not None and comment_field.text else 'N/A'
        outputGroup_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}outputGroup')
        outputGroup = outputGroup_elem.text.strip() if outputGroup_elem is not None and outputGroup_elem.text else 'N/A'
        activityLinkId = exchange.attrib.get('activityLinkId', '').strip()

        # Condition: For intermediateExchange, if amount == 0, delete the row
        if amount == 0.0:
            logger.info(f"{filename}: Skipping intermediateExchange with amount=0.0.")
            return None  # Do not add this record

        # Use activityLinkId to get the corresponding shortname and activityName
        if activityLinkId:
            related_info = global_activity_mapping.get(activityLinkId)
            if not related_info:
                # Try to get from the current file's mapping
                related_info = file_activity_mapping.get(activityLinkId)
            if not related_info:
                related_info = ("Unknown Location", "Unknown Activity Name")
                logger.warning(f"{filename}: activityLinkId '{activityLinkId}' not found in global or file mapping.")
        else:
            # If there is no activityLinkId, use the current file's shortname and activityName
            related_info = (current_location, current_activity_name)

        related_shortname, related_activity_name = related_info

        # Format the flow field
        formatted_flow = f"{flow_name}//[{related_shortname}]{related_activity_name}"

        # Create a record dictionary
        record = {
            'process_name': current_activity_name,
            'flow': formatted_flow,
            'unit': unit_name,
            'amount': amount,
            'category': 'Flow Data',
            'field': 'intermediateExchange',
            'value': '',
            'compartment': '',
            'subcompartment': '',
            'comment': comment,
            'outputGroup': outputGroup,
            'section': 'intermediateExchange',
            'activityLinkId': activityLinkId,
            'intermediateExchangeId': intermediateExchangeId
        }
        return record
    except Exception as e:
        logger.error(f"{filename}: Error processing intermediateExchange - {e}")
        return {}

def process_elementary_exchange(exchange, current_activity_name, filename, file_activity_mapping, namespaces):
    """
    Processes an elementaryExchange element and returns a dictionary record.
    """
    try:
        flow_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}name')
        flow = flow_elem.text.strip() if flow_elem is not None and flow_elem.text else "Unknown Flow"
        unit_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}unitName')
        unit = unit_elem.text.strip() if unit_elem is not None and unit_elem.text else "Unknown Unit"
        amount_str = exchange.attrib.get('amount', 'N/A').strip()
        try:
            amount = float(amount_str)
        except ValueError:
            amount = None
            logger.warning(f"{filename}: Invalid amount value '{amount_str}' in elementaryExchangeId '{exchange.attrib.get('intermediateExchangeId', '').strip()}'.")
    
        comment_field = exchange.find('{http://www.EcoInvent.org/EcoSpold02}comment')
        comment = comment_field.text.strip() if comment_field is not None and comment_field.text else 'N/A'

        # Condition: For elementaryExchange, if amount == 0, delete the row
        if amount == 0.0:
            logger.info(f"{filename}: Skipping elementaryExchange with amount=0.0.")
            return None  # Do not add this record

        # Process compartment and subcompartment
        compartment = exchange.find('{http://www.EcoInvent.org/EcoSpold02}compartment')
        if compartment is not None:
            compartment_main = compartment.find('{http://www.EcoInvent.org/EcoSpold02}compartment')
            compartment_text = compartment_main.text.strip() if compartment_main is not None and compartment_main.text else 'Unknown Compartment'
            subcompartment_elem = compartment.find('{http://www.EcoInvent.org/EcoSpold02}subcompartment')
            subcompartment = subcompartment_elem.text.strip() if subcompartment_elem is not None and subcompartment_elem.text else 'Unknown Subcompartment'
        else:
            compartment_text = 'Unknown Compartment'
            subcompartment = 'Unknown Subcompartment'

        outputGroup_elem = exchange.find('{http://www.EcoInvent.org/EcoSpold02}outputGroup')
        outputGroup = outputGroup_elem.text.strip() if outputGroup_elem is not None and outputGroup_elem.text else 'N/A'
        activityLinkId = exchange.attrib.get('activityLinkId', '').strip()
        intermediateExchangeId = exchange.attrib.get('intermediateExchangeId', '').strip()

        # Use activityLinkId to get the corresponding shortname and activityName
        if activityLinkId:
            related_info = global_activity_mapping.get(activityLinkId)
            if not related_info:
                # Try to get from the current file's mapping
                related_info = file_activity_mapping.get(activityLinkId)
            if not related_info:
                related_info = ("Unknown Location", "Unknown Activity Name")
                logger.warning(f"{filename}: activityLinkId '{activityLinkId}' not found in global or file mapping.")
        else:
            # If there is no activityLinkId, use the current file's shortname and activityName
            related_info = (current_activity_name, current_activity_name)  # It is assumed here that the current activity information is used when there is no activityLinkId

        related_shortname, related_activity_name = related_info

        # Format the flow field
        formatted_flow = f"{flow}//[{related_shortname}]{related_activity_name}"

        # Add compartment and subcompartment in the flow
        formatted_flow = f"{formatted_flow}_{compartment_text}_{subcompartment}"

        # Create a record dictionary
        record = {
            'process_name': current_activity_name,
            'flow': formatted_flow,
            'unit': unit,
            'amount': amount,
            'category': 'Flow Data',
            'field': 'elementaryExchange',
            'value': '',
            'compartment': compartment_text,
            'subcompartment': subcompartment,
            'comment': comment,
            'outputGroup': outputGroup,
            'section': 'elementaryExchange',
            'activityLinkId': activityLinkId,
            'intermediateExchangeId': intermediateExchangeId
        }
        return record
    except Exception as e:
        logger.error(f"{filename}: Error processing elementaryExchange - {e}")
        return {}

def process_file(filename):
    global converted_files, failed_files, failed_file_names
    global deleted_rows_per_file, replaced_process_name, non1_amount_per_file, neg1_amount_per_file, global_activity_mapping, unknown_activity_files

    file_path = os.path.join(input_folder, filename)
    logger.info(f"Processing file {filename}")

    try:
        # Get the file name prefix and suffix
        if "_" in filename:
            prefix, suffix_with_ext = filename.split("_", 1)
            suffix = os.path.splitext(suffix_with_ext)[0].strip()
            logger.info(f"{filename}: Filename split into prefix: '{prefix}' and suffix: '{suffix}'")
        else:
            prefix = os.path.splitext(filename)[0].strip()
            suffix = ""
            logger.warning(f"{filename}: Filename does not contain '_'. Using entire name as prefix.")

        # Get activityName and location
        file_prefix = prefix  # The prefix has already been extracted
        process_info = lookup_dict.get(file_prefix, ("Unknown Activity", "Unknown Location"))
        current_activity_name, current_location = process_info
        logger.info(f"{filename}: Current activity: '{current_activity_name}', Location: '{current_location}'")

        # Initialize an empty list to store all records
        records = []

        # Initialize delete count
        deleted_rows_per_file[filename] = 0

        # Initialize non-1 amount count
        non1_amount_per_file[filename] = 0

        # Initialize -1 amount count
        neg1_amount_per_file[filename] = 0

        # Parse the .spold file
        ecoSpold = parse_file_v2(file_path)
        namespaces = {'eco': 'http://www.EcoInvent.org/EcoSpold02'}

        # Extract the activity mapping of the current file
        file_activity_mapping = extract_activity_mapping(file_path)

        # Extract information from the activityDescription section
        activityDescription = ecoSpold.find('.//eco:activityDescription', namespaces)
        activity_records = process_activity_description(activityDescription, current_activity_name, namespaces)
        records.extend(activity_records)
        logger.info(f"{filename}: Extracted {len(activity_records)} activityDescription records.")

        # Extract information from the intermediateExchange section
        intermediate_exchanges = ecoSpold.findall('.//eco:intermediateExchange', namespaces)
        logger.info(f"{filename}: Found {len(intermediate_exchanges)} intermediateExchange elements.")

        # Identify all intermediateExchangeIds that match the suffix
        if suffix:
            matching_exchanges = [exchange for exchange in intermediate_exchanges if exchange.attrib.get('intermediateExchangeId', '').strip() == suffix]
            logger.info(f"{filename}: Found {len(matching_exchanges)} intermediateExchange elements matching suffix '{suffix}'.")
        else:
            matching_exchanges = []
            logger.warning(f"{filename}: Suffix is empty. No intermediateExchangeId to match.")

        # Update activityLinkId according to logic
        if len(matching_exchanges) == 1:
            exchange_to_update = matching_exchanges[0]
            exchange_to_update.attrib['activityLinkId'] = prefix
            logger.info(f"{filename}: Only one matching intermediateExchangeId. Updated activityLinkId to '{prefix}'.")
        elif len(matching_exchanges) > 1:
            # Update all activityLinkIds with amount=1 or amount=-1
            # First find all amount=1 or 1.0
            amount_1_exchanges = [ex for ex in matching_exchanges if ex.attrib.get('amount', '').strip() in ['1', '1.0']]
            if amount_1_exchanges:
                for ex in amount_1_exchanges:
                    ex.attrib['activityLinkId'] = prefix
                    logger.info(f"{filename}: Updated activityLinkId to '{prefix}' for intermediateExchangeId '{ex.attrib.get('intermediateExchangeId')}' with amount={ex.attrib.get('amount')}.")
            else:
                # If there is no amount=1, then find amount=-1 or -1.0
                amount_neg1_exchanges = [ex for ex in matching_exchanges if ex.attrib.get('amount', '').strip() in ['-1', '-1.0']]
                if amount_neg1_exchanges:
                    for ex in amount_neg1_exchanges:
                        ex.attrib['activityLinkId'] = prefix
                        logger.info(f"{filename}: Updated activityLinkId to '{prefix}' for intermediateExchangeId '{ex.attrib.get('intermediateExchangeId')}' with amount={ex.attrib.get('amount')}.")
                else:
                    logger.info(f"{filename}: Multiple matches. No amount=1 or amount=-1. No changes made.")

        # Process all intermediateExchanges
        for exchange in intermediate_exchanges:
            record = process_intermediate_exchange(exchange, current_activity_name, current_location, filename, file_activity_mapping, namespaces)
            if record:
                records.append(record)
            else:
                deleted_rows_per_file[filename] += 1

            # Count the cases where activityLinkId is missing and amount != 1
            activityLinkId = exchange.attrib.get('activityLinkId', '').strip()
            amount_str = exchange.attrib.get('amount', 'N/A').strip()
            try:
                amount = float(amount_str)
            except ValueError:
                amount = None
                logger.warning(f"{filename}: Invalid amount value '{amount_str}' in intermediateExchangeId '{exchange.attrib.get('intermediateExchangeId', '').strip()}'.")
        
            if not activityLinkId and amount is not None and amount != 1.0:
                non1_amount_per_file[filename] = non1_amount_per_file.get(filename, 0) + 1
                logger.info(f"{filename}: Found intermediateExchange with amount={amount} and no activityLinkId.")
                if amount == -1.0:
                    neg1_amount_per_file[filename] = neg1_amount_per_file.get(filename, 0) + 1
                    logger.info(f"{filename}: Found intermediateExchange with amount=-1.0 and no activityLinkId.")

        # Extract information from the elementaryExchange section
        elementary_exchanges = ecoSpold.findall('.//eco:elementaryExchange', namespaces)
        logger.info(f"{filename}: Found {len(elementary_exchanges)} elementaryExchange elements.")
        for exchange in elementary_exchanges:
            record = process_elementary_exchange(exchange, current_activity_name, filename, file_activity_mapping, namespaces)
            if record:  # Add only when processing is successful
                records.append(record)
            else:
                deleted_rows_per_file[filename] += 1

            # Count the cases where activityLinkId is missing and amount != 1
            activityLinkId = exchange.attrib.get('activityLinkId', '').strip()
            amount_str = exchange.attrib.get('amount', 'N/A').strip()
            try:
                amount = float(amount_str)
            except ValueError:
                amount = None
                logger.warning(f"{filename}: Invalid amount value '{amount_str}' in elementaryExchangeId '{exchange.attrib.get('intermediateExchangeId', '').strip()}'.")
    
            if not activityLinkId and amount is not None and amount != 1.0:
                non1_amount_per_file[filename] = non1_amount_per_file.get(filename, 0) + 1
                logger.info(f"{filename}: Found elementaryExchange with amount={amount} and no activityLinkId.")
                if amount == -1.0:
                    neg1_amount_per_file[filename] = neg1_amount_per_file.get(filename, 0) + 1
                    logger.info(f"{filename}: Found elementaryExchange with amount=-1.0 and no activityLinkId.")

        # Create DataFrame
        all_data = pd.DataFrame(records, columns=[
            'process_name', 'flow', 'unit', 'amount', 'category', 'field', 'value',
            'compartment', 'subcompartment', 'comment', 'outputGroup', 'section',
            'activityLinkId', 'intermediateExchangeId'
        ])

        # Generate a combined string and check if it matches "prefix_suffix"
        matched_flow = None
        target_combined = f"{prefix}_{suffix}"
        logger.info(f"{filename}: Target combined string: '{target_combined}'")

        # Create a boolean condition
        condition = (all_data['activityLinkId'].fillna('') + '_' + all_data['intermediateExchangeId'].fillna('')) == target_combined

        if any(condition):
            matched_flow = all_data.loc[condition, 'flow'].iloc[0]  # Assume there is only one match
            logger.info(f"{filename}: Matched combined string '{target_combined}'. Setting process_name to '{matched_flow}' for all records.")

        # Replace the process_name of the entire file based on the match result
        if matched_flow:
            all_data['process_name'] = matched_flow  # Replace the entire column
            replaced_process_name[filename] = matched_flow
            logger.info(f"{filename}: process_name has been set to '{matched_flow}' for all records based on combination '{target_combined}'.")

        # Check if the flow field contains [Unknown Location]Unknown Activity Name
        if any(all_data['flow'].str.contains(r'\[Unknown Location\]Unknown Activity Name', na=False)):
            unknown_activity_files.append(filename)
            logger.warning(f"{filename}: Contains '[Unknown Location]Unknown Activity Name' in flow fields after processing.")

        # Select only records of the 'Flow Data' category and create a copy
        flow_data = all_data[all_data['category'] == 'Flow Data'].copy()
        logger.info(f"{filename}: 'Flow Data' records count: {len(flow_data)}")

        # **New feature: Modify the content of the 'flow' column that meets the conditions**
        # Define the function to modify the flow
        def modify_flow(flow):
            """
            Modify the content of the flow field:
            If the flow contains "//", delete all content from "//" to the first "_"
            and add a "_" between the remaining parts.
            """
            if '//' in flow:
                try:
                    start = flow.index('//')
                    # Find the first '_' after the start
                    after_start = flow[start:].index('_')
                    end = start + after_start + 1  # Include '_'
                    # Delete the content from "//" to before the first "_" and add "_"
                    modified_flow = flow[:start] + '_' + flow[end:]
                    logger.debug(f"Original flow: '{flow}' | Modified flow: '{modified_flow}'")
                    return modified_flow
                except ValueError:
                    # If '_' is not found, return the original flow
                    logger.warning(f"Flow '{flow}' contains '//' but no '_'. No modification applied.")
                    return flow
            else:
                return flow

        # Apply the modify_flow function to the rows in the 'flow' column that meet the conditions
        try:
            # Apply modification only to rows with 'elementaryExchange'
            condition_modify = flow_data['field'] == 'elementaryExchange'
            original_count = flow_data[condition_modify].shape[0]
            flow_data.loc[condition_modify, 'flow'] = flow_data.loc[condition_modify, 'flow'].apply(modify_flow)
            modified_count = flow_data[condition_modify]['flow'].str.contains('_').sum()  # A simple check to see if the modification was successful
            logger.info(f"{filename}: Modified 'flow' column for 'elementaryExchange' records. {modified_count}/{original_count} records modified.")
        except Exception as e:
            logger.error(f"{filename}: Error modifying 'flow' column for 'elementaryExchange' - {e}")

        # **End of new feature**

        # **New feature: Delete duplicate rows where outputGroup is 'N/A'**
        # Find rows that are identical except for 'outputGroup'
        subset_cols = [col for col in flow_data.columns if col != 'outputGroup']
        duplicates = flow_data.duplicated(subset=subset_cols, keep=False)

        duplicated_flow_data = flow_data[duplicates]

        # Initialize a list to save the indexes to be deleted
        indexes_to_drop = []

        # Group and find duplicate rows
        grouped = duplicated_flow_data.groupby(subset_cols)

        for group_keys, group in grouped:
            if set(group['outputGroup']) == {'0', 'N/A'}:
                # Find the rows where outputGroup is 'N/A' and mark them for deletion
                n_a_rows = group[group['outputGroup'] == 'N/A']
                indexes_to_drop.extend(n_a_rows.index.tolist())

        if indexes_to_drop:
            flow_data = flow_data.drop(indexes_to_drop)
            logger.info(f"{filename}: Removed {len(indexes_to_drop)} duplicate rows with outputGroup 'N/A'.")

        # **End of new feature**

        # Generate the output file path, use os.path.splitext to ensure the correct extension is replaced
        output_filename = f"{os.path.splitext(filename)[0]}.csv"
        output_path = os.path.join(batch_folder, output_filename)

        # Save 'Flow Data' related records to a CSV file
        try:
            # Even if flow_data is empty, generate a CSV file with a header
            flow_data.to_csv(output_path, index=False, encoding='utf-8')
            converted_files += 1
            logger.info(f"{filename}: Flow data processed and saved to '{output_path}'")
        except Exception as e:
            failed_files += 1
            failed_file_names.append(filename)
            logger.error(f"{filename}: Error saving CSV file - {e}")

    except Exception as e:
        logger.error(f"{filename}: Error processing file - {e}")
        failed_files += 1
        failed_file_names.append(filename)

def main():
    global total_files, converted_files, failed_files, failed_file_names

    # First pass, traverse all files to build a global map
    logger.info("Starting first pass to build global activity mapping.")
    for filename in os.listdir(input_folder):
        if filename.endswith(".spold"):
            file_path = os.path.join(input_folder, filename)
            logger.info(f"Building mapping from file {filename}")
            file_mapping = extract_activity_mapping(file_path)
            global_activity_mapping.update(file_mapping)

    logger.info(f"Global activity mapping built with {len(global_activity_mapping)} entries.")

    # Output the global mapping dictionary to a CSV file for inspection
    mapping_output_path = os.path.join(batch_folder, "global_activity_mapping.csv")
    try:
        with open(mapping_output_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['activity_id', 'shortname', 'activityName'])
            for activity_id, (shortname, activityName) in global_activity_mapping.items():
                writer.writerow([activity_id, shortname, activityName])
        logger.info(f"Global activity mapping exported to {mapping_output_path}")
    except Exception as e:
        logger.error(f"Error exporting global activity mapping to CSV: {e}")

    # Second pass, traverse all files to process data
    logger.info("Starting second pass to process all files.")

    for filename in os.listdir(input_folder):
        if filename.endswith(".spold"):
            total_files += 1
            logger.info(f"Processing file {filename}")

            try:
                process_file(filename)
            except Exception as e:
                failed_files += 1
                failed_file_names.append(filename)
                logger.error(f"{filename}: Error processing file - {e}")
                continue

    # Generate a list of failed files and save it to a file
    failed_files_log_path = os.path.join(batch_folder, "failed_files.txt")
    try:
        with open(failed_files_log_path, 'w', encoding='utf-8') as f:
            for fname in failed_file_names:
                f.write(f"{fname}\n")
        logger.info(f"Failed file names saved to {failed_files_log_path}")
    except Exception as e:
        logger.error(f"Error writing failed files log: {e}")

    # Generate a processing summary and save it to summary.txt,
    # including statistics on the number of deleted rows and process_name replacements
    summary_log_path = os.path.join(batch_folder, "summary.txt")
    try:
        with open(summary_log_path, 'w', encoding='utf-8') as f:
            f.write("=== Processing Summary ===\n")
            f.write(f"Total .spold files found: {total_files}\n")
            f.write(f"Successfully converted to CSV: {converted_files}\n")
            f.write(f"Failed to convert: {failed_files}\n\n")
            if failed_files > 0:
                f.write("List of failed files:\n")
                for fname in failed_file_names:
                    f.write(f"- {fname}\n")
            f.write("\n=== Deleted Rows Per File ===\n")
            for fname, count in deleted_rows_per_file.items():
                f.write(f"{fname}: {count} rows deleted\n")
            f.write("\n=== Replaced Process Name Per File ===\n")
            for fname, flow in replaced_process_name.items():
                f.write(f"{fname}: process_name replaced with flow '{flow}'\n")
            f.write("\n=== Non-1 Amount Without activityLinkId Per File ===\n")
            for fname, count in non1_amount_per_file.items():
                f.write(f"{fname}: {count} rows with amount != 1 and no activityLinkId\n")
            f.write("\n=== Amount=-1 Without activityLinkId Per File ===\n")
            for fname, count in neg1_amount_per_file.items():
                f.write(f"{fname}: {count} rows with amount=-1 and no activityLinkId\n")
            f.write("\n=== Files with Unknown Activity Name After Deletion ===\n")
            for fname in unknown_activity_files:
                f.write(f"{fname}\n")
        logger.info(f"Processing summary saved to {summary_log_path}")
    except Exception as e:
        logger.error(f"Error writing summary log: {e}")

    # Save non1_amount_per_file to CSV
    output_non1_path = os.path.join(batch_folder, "non1_amount_files.csv")
    try:
        with open(output_non1_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Filename', 'Non-1 Amount Count'])
            for fname, count in non1_amount_per_file.items():
                if count > 0:
                    writer.writerow([fname, count])
        logger.info(f"Non-1 amount files saved to {output_non1_path}")
    except Exception as e:
        logger.error(f"Error exporting non1 amount files to CSV: {e}")

    # Save neg1_amount_per_file to CSV
    output_neg1_path = os.path.join(batch_folder, "neg1_amount_files.csv")
    try:
        with open(output_neg1_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Filename', 'Amount=-1 Count'])
            for fname, count in neg1_amount_per_file.items():
                if count > 0:
                    writer.writerow([fname, count])
        logger.info(f"Amount=-1 files saved to {output_neg1_path}")
    except Exception as e:
        logger.error(f"Error exporting amount=-1 files to CSV: {e}")

    # Tally the results and record them in summary.txt
    logger.info("=== Processing Summary ===")
    logger.info(f"Total .spold files found: {total_files}")
    logger.info(f"Successfully converted to CSV: {converted_files}")
    logger.info(f"Failed to convert: {failed_files}")

    if failed_files > 0:
        logger.info("List of failed files:")
        for fname in failed_file_names:
            logger.info(f"- {fname}")

    # Record the number of deleted rows for each file
    logger.info("=== Deleted Rows Per File ===")
    for fname, count in deleted_rows_per_file.items():
        logger.info(f"{fname}: {count} rows deleted")

    # Record the replaced process_name for each file
    logger.info("=== Replaced Process Name Per File ===")
    for fname, flow in replaced_process_name.items():
        logger.info(f"{fname}: process_name replaced with flow '{flow}'")

    # Record the number of rows where activityLinkId is missing and amount != 1
    logger.info("=== Non-1 Amount Without activityLinkId Per File ===")
    for fname, count in non1_amount_per_file.items():
        logger.info(f"{fname}: {count} rows with amount != 1 and no activityLinkId")

    # Record the number of rows where activityLinkId is missing and amount == -1
    logger.info("=== Amount=-1 Without activityLinkId Per File ===")
    for fname, count in neg1_amount_per_file.items():
        logger.info(f"{fname}: {count} rows with amount=-1 and no activityLinkId")

    # Record files that still have [Unknown Location]Unknown Activity Name
    logger.info("=== Files with Unknown Activity Name After Deletion ===")
    for fname in unknown_activity_files:
        logger.info(f"{fname}")

    # Increment the batch number for the next run
    try:
        with open(batch_file_path, "w") as f:
            f.write(str(batch_number + 1))
        logger.info(f"Batch number incremented to {batch_number + 1}")
    except Exception as e:
        logger.error(f"Error updating batch number: {e}")

    logger.info("All files processed successfully.")

if __name__ == "__main__":
    main()

In [ ]:
import os

def compare_directories(dir1, dir2):
    """
    Compare two directories and find files present in dir2 but missing in dir1.
    
    Args:
        dir1 (str): The path to the first directory.
        dir2 (str): The path to the second directory.
    
    Returns:
        list: A list of files present in dir2 but missing in dir1.
    """
    # List files in both directories
    files_dir1 = set(os.listdir(dir1))
    files_dir2 = set(os.listdir(dir2))
    
    # Find files present in dir2 but missing in dir1
    missing_files = files_dir2 - files_dir1
    return list(missing_files)

if __name__ == "__main__":
    # Define the directories to compare
    dir1 = r"C:\Users\WasteWang\LCA\OUTPUT\1213_93"
    dir2 = r"C:\Users\WasteWang\LCA\OUTPUT\1211_88"
    
    # Compare directories
    missing_files = compare_directories(dir1, dir2)
    
    # Print results
    if missing_files:
        print("Files present in dir2 but missing in dir1:")
        for file in missing_files:
            print(file)
    else:
        print("No files are missing in dir1 compared to dir2.")


In [ ]:
import os
import pandas as pd
from collections import OrderedDict

def build_lca_matrix(folder_path, output_file):
    # A list to record process_names in the order they are processed
    process_name_list = []
    # Initialize an ordered dictionary to maintain insertion order
    data_dict = OrderedDict()
    flow_set = set()

    # List all CSV file paths
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    total_files = len(csv_files)
    print(f"There are a total of {total_files} CSV files to process.")

    # Used to record files that were not processed successfully
    failed_files = []
    # Used to detect duplicate process_names
    process_name_set = set()
    duplicate_process_files = []

    for idx, file_name in enumerate(csv_files, 1):
        file_path = os.path.join(folder_path, file_name)
        try:
            # Try to read with UTF-8
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            # If UTF-8 fails, try gbk
            try:
                df = pd.read_csv(file_path, encoding='gbk')
            except Exception as e:
                print(f"Error: Could not read file {file_name}. Reason: {e}")
                failed_files.append(file_name)
                continue
        except Exception as e:
            print(f"Error: Could not read file {file_name}. Reason: {e}")
            failed_files.append(file_name)
            continue

        # Check for required columns
        if not {'process_name', 'field', 'flow', 'amount'}.issubset(df.columns):
            print(f"Warning: File {file_name} is missing necessary columns.")
            failed_files.append(file_name)
            continue

        process_names = df['process_name'].unique()
        if len(process_names) < 1:
            print(f"Warning: Could not find a process_name in file {file_name}.")
            failed_files.append(file_name)
            continue

        process_name = process_names[0]
        if len(process_names) > 1:
            print(f"Warning: Multiple process_names exist in file {file_name}. The first one will be used: {process_name}")

        # Check for duplicate process_names
        if process_name in process_name_set:
            print(f"Warning: Duplicate process_name '{process_name}' found in file {file_name}, skipping this file.")
            duplicate_process_files.append(file_name)
            failed_files.append(file_name)
            continue
        else:
            process_name_set.add(process_name)
            # Add the process_name to the list in the order it was processed
            process_name_list.append(process_name)

        # Filter for the required fields
        df_filtered = df[df['field'].isin(['intermediateExchange', 'elementaryExchange'])]

        if df_filtered.empty:
            print(f"Warning: No valid rows after filtering in file {file_name}.")
            failed_files.append(file_name)
            continue

        # Separate intermediate and elementary exchanges
        df_intermediate = df_filtered[df_filtered['field'] == 'intermediateExchange']
        df_elementary = df_filtered[df_filtered['field'] == 'elementaryExchange']

        if df_intermediate.empty and df_elementary.empty:
            print(f"Warning: No intermediateExchange or elementaryExchange rows in file {file_name}.")
            failed_files.append(file_name)
            continue

        column_added = False

        # Process intermediateExchange
        for _, row in df_intermediate.iterrows():
            flow = row['flow']
            amount = row['amount'] if pd.notnull(row['amount']) else 0
            if flow not in flow_set:
                flow_set.add(flow)
                data_dict[flow] = {}
            data_dict[flow][process_name] = amount
            column_added = True

        # Process elementaryExchange
        for _, row in df_elementary.iterrows():
            flow = row['flow']
            amount = row['amount'] if pd.notnull(row['amount']) else 0
            if flow not in flow_set:
                flow_set.add(flow)
                data_dict[flow] = {}
            data_dict[flow][process_name] = amount
            column_added = True

        if not column_added:
            failed_files.append(file_name)

        if idx % 1000 == 0 or idx == total_files:
            print(f"Processed {idx}/{total_files} files.")

    # Merge data to build the matrix
    print("Building DataFrame...")
    all_flows = list(data_dict.keys())
    matrix_df = pd.DataFrame.from_dict(data_dict, orient='index').fillna(0)
    # Sort rows by insertion order
    matrix_df = matrix_df.reindex(all_flows)
    # Use process_name_list to ensure column order
    matrix_df = matrix_df.reindex(columns=process_name_list)
    matrix_df.index.name = 'flow'

    # Diagonalization process:
    # If a column name exists in the row names, move the corresponding row
    # to the same index as the column (on the diagonal).
    row_order = list(matrix_df.index)
    for i, col in enumerate(matrix_df.columns):
        if col in row_order:
            current_pos = row_order.index(col)
            if current_pos != i:
                # Swap rows so that (col, col) is on the diagonal
                row_order[i], row_order[current_pos] = row_order[current_pos], row_order[i]
    matrix_df = matrix_df.reindex(index=row_order)

    # Save to CSV
    print(f"Saving matrix to {output_file}...")
    matrix_df.to_csv(output_file, encoding='utf-8')
    print("Matrix construction complete!")

    # Statistics
    print("\n===== Matrix Statistics =====")
    num_rows, num_cols = matrix_df.shape
    total_elements = num_rows * num_cols
    non_zero = (matrix_df != 0).sum().sum()
    non_zero_ratio = non_zero / total_elements if total_elements != 0 else 0
    negative = (matrix_df < 0).sum().sum()
    negative_ratio = negative / total_elements if total_elements != 0 else 0
    negative_over_non_zero = (negative / non_zero) if non_zero != 0 else 0

    print(f"Number of rows: {num_rows}")
    print(f"Number of columns: {num_cols}")
    print(f"Number of non-zero values: {non_zero}")
    print(f"Percentage of non-zero values: {non_zero_ratio:.2%}")
    print(f"Number of negative values: {negative}")
    print(f"Percentage of negative values: {negative_ratio:.2%}")
    print(f"Proportion of negative values among non-zero values: {negative_over_non_zero:.2%}")
    print("========================\n")

    # Count the number of all-zero rows
    all_zero_rows = (matrix_df.sum(axis=1) == 0).sum()
    print(f"Number of all-zero rows: {all_zero_rows}")

    # Output the list of files that did not generate a new column
    if failed_files:
        failed_file_path = os.path.join(os.path.dirname(output_file), "failed_files.txt")
        with open(failed_file_path, "w", encoding="utf-8") as f:
            f.write("List of CSV files that did not generate new columns:\n")
            for file in failed_files:
                f.write(f"{file}\n")
        print(f"\nThe following CSV files did not generate new columns. See '{failed_file_path}' for details:")
        for file in failed_files:
            print(f" - {file}")
    else:
        print("All CSV files have successfully generated new columns.")

    # Output CSV files with duplicate process_names
    if duplicate_process_files:
        duplicate_file_path = os.path.join(os.path.dirname(output_file), "duplicate_process_files.txt")
        with open(duplicate_file_path, "w", encoding='utf-8') as f:
            f.write("List of CSV files with duplicate process_names:\n")
            for file in duplicate_process_files:
                f.write(f"{file}\n")
        print(f"\nThe following CSV files have duplicate process_names and did not generate new columns. See '{duplicate_file_path}' for details:")
        for file in duplicate_process_files:
            print(f" - {file}")
    else:
        print("No duplicate process_names were found.")

if __name__ == "__main__":
    folder_path = r"C:\Users\WasteWang\LCA\OUTPUT\1213_93"
    output_file = r"C:\Users\WasteWang\LCA\OUTPUT\NEW_3.11_1213_93_LCA_matrix.csv"
    # The filename needs to be changed for each run
    build_lca_matrix(folder_path, output_file)